In [1]:
# Add client package to sys path

MLAPI_PATH = '/home/jovyan/mlapi'

import sys

if MLAPI_PATH not in sys.path:
    sys.path.append(MLAPI_PATH)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data = {
    'name': 'sample',
    'algorithm': 'seq2seq',
    'forecast_horizon': 10,
    'perform_hpo': False,
    'bucket_name': 'sample'
}

In [4]:
import requests

In [5]:
url = 'http://mlapi:80/forecast'
response = requests.post(url, json=data)

In [7]:
response

<Response [200]>

In [8]:
from requests import Session

In [9]:
s = Session()

In [11]:
s.post(url, data=data)

<Response [422]>